In [1]:
import rpy2

%load_ext rpy2.ipython

In [2]:
%%R
library(tidyverse)
library(ggpubr)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [3]:
%%R
df20 <- read_delim('../results/combined/all.csv', delim=',') %>% group_by(genus,species) %>% mutate(recomb_bin = ntile(cm_per_mb, 20))
pg_namedf <- read_delim('../data/genus_phylogenetic_group_metadata.txt', delim='\t')
merge_pg_name <- left_join(df20,pg_namedf, by = c('genus'='genus'))
merge_pg_name$pg_name <- as_factor(merge_pg_name$pg_name)
nes <- read_delim('../results/model/Nested_model/Ne_Pi_recomb.csv', delim = ',') %>% select('genus','species','NE_MEAN','NE_MEDIAN')
nes <- nes %>% 
    group_by(genus,species) %>%
    reframe(
        NE_MEAN = mean(NE_MEAN),
        NE_MEDIAN = mean(NE_MEDIAN))

Rows: 5257244 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): chr, species, genus
dbl (13): start, end, freq_mean, freq_min, freq_max, freq_median, window_siz...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 57 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): genus, pg_name

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 5071829 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): genus, species, FAMILY, chr
dbl (21): ...1, Unnamed: 0, MEAN, MEDIAN, VARIANCE, NE_MEAN, NE_MEDIAN, NE_V...

ℹ Use `spec()` to retrieve the full column specifi

In [4]:
%%R
merged_tibble <- left_join(nes,tester, by = c('species' = 'species', 'genus'='genus'))
merged_tibble

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (1309648289.py, line 1)

In [ ]:
%%R
pi_log_r_example <- merged_tibble %>%  
    filter(genus %in% c('Pongo','Eulemur','Macaca','Pithecia') & species %in% c('abelii','fulvus','fascicularis','albicans')) %>%
    ggplot(aes(x = log10(cm_per_mb), y = mean_pi, group = species))+
    geom_point(aes(fill=species),color = 'black', size = 3, alpha = 1, shape = 21, stroke = 1)+
    geom_smooth(method = 'lm', se = F, aes( col = species))+
    scale_color_manual(values = c('#240a34','#891652','#eabe6c','#db7552'))+
    scale_fill_manual(values = c('#240a34','#891652','#eabe6c','#db7552'))+
    labs(x = "Log10 Recombination Rate (cM/mb)", y = "Average pairwise differences (π)", title = "", size = 55) +
    ggpubr::theme_pubr(legend = 'none') +
    scale_x_continuous(expand = c(0, 0))+
    scale_y_continuous(expand = expansion(mult = c(0,0)))
pi_log_r_example

In [ ]:
%%R
z_scores_pi <- merged_tibble %>% 
    ggplot(aes(x = z_cm, y = z_pi, group = species, col = log10(NE_MEDIAN)))+
    #geom_point()+
    geom_line(alpha = .4, size =1.5)+
    theme(legend.position = 'none')+
    scale_colour_viridis_c(name = "Effective population size")+
    #scale_color_continuous(name = "Effective population size", low = "#ffff33", high = "#e41a1c",
    #                       breaks = log10(c(10000,30000, 100000, 300000)),
    #                       labels = c('10','30',"100",'300'))+
    labs(x = "Z score of Recombination Rate", y = "Z score of π", title = "", size = 55) +
    ggpubr::theme_pubr() +
    #theme(legend.position = 'none')+
    scale_x_continuous(expand = c(0, 0)) +
    scale_y_continuous(expand = expansion(mult = c(0,0)))

In [ ]:
%%R -h 800 -w 800
z_scores_pi_log_cm <- merged_tibble %>% 
    ggplot(aes(x = log10(cm_per_mb), y = z_pi, group = species, col = log10(NE_MEDIAN)))+
    geom_point()+
    geom_line(alpha = 2, size =1.5)+
    theme(legend.position = 'none')+
    #scale_colour_viridis_c(name = "Effective population size
    scale_color_continuous(name = "Effective population size", low = "#891652", high = "#eabe6c",
                           breaks = log10(c(10000,30000, 100000, 300000)),
                           labels = c('10','30',"100",'300'))+
    labs(x = "log10 Recombination Rate", y = "Z score of π", title = "", size = 55) +
    ggpubr::theme_pubr() #+
    #theme(legend.position = 'none')+ "#9C964A", "#CDC08C"
   # scale_x_continuous(expand = c(0, 0)) +
    #scale_y_continuous(expand = expansion(mult = c(0,0)))
print(z_scores_pi_log_cm)

In [ ]:
%%R 
library(tidyverse)
a <- read_delim('../../results/model/Nested_model/log_ne_log_cm_sd_pi/a_24_04_2024.tsv', delim="\t")
b <- read_delim('../../results/model/Nested_model/log_ne_log_cm_sd_pi/b_24_04_2024.tsv', delim="\t")
h0 <- read_delim('../../results/model/Nested_model/log_ne_log_cm_sd_pi/h0_24_04_2024.tsv', delim="\t")
h1 <- read_delim('../../results/model/Nested_model/log_ne_log_cm_sd_pi/h1_24_04_2024.tsv', delim ="\t")
Ne <- read_csv('../../results/model/Nested_model/log_scaled_z_scores.csv') %>% select(full_species,NE_MEDIAN,z_ne) %>% distinct(full_species, .keep_all=TRUE) %>% separate(full_species, into = c("genus", "species"), sep = "_", remove = FALSE)
pg_namedf <- read_delim('../../data/genus_phylogenetic_group_metadata.txt', delim='\t')

In [ ]:
%%R
# Define parameters
sample <- 1000
h0_vec <- sample(h0$h0, sample, replace = TRUE)
h1_vec <- sample(h1$h1, sample, replace = TRUE)

# Generate sampled points
sampled_points <- map_dfr(1:sample, ~{
  alpha <- sample(h0_vec, 1)
  beta <- sample(h1_vec, 1)
  tibble(x = seq(4000, 5000000, by = 1000), y = alpha + beta * log10(x), logx = log10(x))
})

sampled_points$x <- as.character(sampled_points$x)

HPDI_h1_ne <- sampled_points %>% 
    group_by(x,logx) %>%
    reframe(
        fifty_low = quantile(y,0.25),
        fifty_max = quantile(y,0.75),
        seventyfive_low = quantile(y,0.125),
        seventyfive_max = quantile(y,0.875),
        ninetyfive_low = quantile(y,0.025),
        ninetyfive_max = quantile(y,0.975)
        )
HPDI_h1_ne$x <- as.double(HPDI_h1_ne$x)
HPDI_h1_ne

In [ ]:
%%R
b_species <- b %>% 
    group_by(Species) %>% reframe(
    upper = quantile(b,.95),
    lower = quantile(b,.05),
    b = mean(b))
b_species
merged_data <- merge(b_species, Ne, by.x = "Species", by.y = "full_species", all.x = TRUE) 
merged_data <- merge(merged_data, pg_namedf, by.x = "genus", by.y = "genus", all.x = TRUE)

In [ ]:
%%R
# Calculate the posterior predictive distribution at a given x
posterior_predictive <- function(x, slope_samples, intercept_samples) {
  slope_samples * x + intercept_samples
}

# Calculate the credibility interval
credibility_interval <- function(x, slope_samples, intercept_samples, prob = c(0.025, 0.975)) {
  quantile(posterior_predictive(x, slope_samples, intercept_samples), prob)
}

# Use the function
x_values <- c()
lower_bounds <- c()
upper_bounds <- c()

# Calculate the credibility intervals for each x value
for (i in log10(merged_data$NE_MEDIAN)) {
    cred_int <- credibility_interval(i, h1$h1, h0$h0)
    x_values <- c(x_values, i)
    lower_bounds <- c(lower_bounds, cred_int[1])
    upper_bounds <- c(upper_bounds, cred_int[2])
}

# Create a data frame
mean_reg_line <- data.frame(
    x = x_values,
    lower_bound = lower_bounds,
    upper_bound = upper_bounds
)


In [ ]:
%%R -w 1000 -h 1000
fig3_par1 <- merged_data %>% 
    ggplot()+
    geom_errorbar(aes(x = log10(NE_MEDIAN), ymin=lower, ymax=upper), col = 'black', width=.005,
                     position=position_dodge(.9)) +
    geom_point(aes(x=log10(NE_MEDIAN),y=b,fill=pg_name), color = 'black', size = 3, alpha = 1, shape = 21, stroke = 1)+
    geom_ribbon(data=mean_reg_line, aes(x= x, ymin = lower_bound, ymax = upper_bound), fill='grey', alpha = .5)+
    geom_line(aes(x=log10(NE_MEDIAN), y=(mean(h0$h0))+(mean(h1$h1)*log10(NE_MEDIAN))), lty = 2, size =2)+
    scale_fill_manual(values=c('#240a34','#891652','#eabe6c','#db7552')) +
     ylab("Impact of Linked Selection")+
     xlab('Log10 Effecive population size')+
    scale_x_continuous(expand = c(0, 0), limits = c(4,5.6))+
    scale_y_continuous(expand = expansion(mult = c(0,0)))+
    ggpubr::theme_pubr(legend = 'none')
print(fig3_par1)

In [ ]:
%%R
fig3 <- ggarrange(
  ggarrange(z_scores_pi_log_cm, pi_log_r_example, ncol = 1), # First column with two plots stacked vertically
  fig3_par1,                                             # Second column with one plot
    
  ncol = 2                                           # Two columns in total
)
print(fig3)

In [ ]:
%%R


# Save the plot
ggsave('figure3_impact_model.svg', plot = fig3)